# <span style="font-width:bold; font-size: 3rem; color:#1EB182;">**Hopsworks Feature Store** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Model training & UI Exploration</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/citibike/4_model_training_and_registration.ipynb)


## 🗒️ This notebook is divided into 3 main sections:
1. **Loading the training data**.
2. **Train the model**.
3. **Register model to Hopsworks model registry**.

![tutorial-flow](https://github.com/logicalclocks/hopsworks-tutorials/blob/master/images/03_model.png?raw=1)

### <span style="color:#ff5f27;">📝 Importing Libraries</span>

In [ ]:
!pip install -U hopsworks --quiet

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import xgboost as xgb

from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings('ignore')

---

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

---

## <span style="color:#ff5f27;">🪝 Feature View and Training Dataset Retrieval</span>

In [ ]:
feature_view = fs.get_feature_view(
    name = 'citibike_fv',
    version = 1
)

In [ ]:
X, _ = feature_view.get_training_data(1)

In [ ]:
X.head(3)

In [ ]:
X.iloc[:, 1:-1] = X.iloc[:, 1:-1].astype(float)

In [ ]:
X.shape

In [ ]:
y = X.pop("users_count")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [ ]:
X_train = X_train.set_index(["date", "station_id"])
X_test = X_test.set_index(["date", "station_id"])

In [ ]:
X_train.shape, X_test.shape

In [ ]:
X_train.head(3)

---

## <span style="color:#ff5f27;">🧬 Modeling</span>

In [ ]:
regressor = xgb.XGBRegressor()
 
# Fitting the model
regressor.fit(X_train, y_train)
 
# Predict the model
pred_xgb = regressor.predict(X_test)
 
r2_xgb = r2_score(pred_xgb, y_test.values)
print("R2 score for XGBoost model:", r2_xgb)

---

## <span style='color:#ff5f27'>🗄 Model Registry</span>

One of the features in Hopsworks is the model registry. This is where you can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

In [ ]:
mr = project.get_model_registry()

### <span style="color:#ff5f27;">⚙️ Model Schema</span>

The model needs to be set up with a [Model Schema](https://docs.hopsworks.ai/3.0/user_guides/mlops/registry/model_schema/), which describes the inputs and outputs for a model.

A Model Schema can be automatically generated from training examples, as shown below.

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

In [ ]:
import joblib

joblib.dump(regressor, 'citibike_xgb_model.pkl')

In [ ]:
model = mr.sklearn.create_model(
    name="citibike_xgb_model",
    version=1,
    metrics={"r2_score": r2_xgb},
    description="XGBoost regressor. Citibike Project.",
    input_example=X_train.sample(),
    model_schema=model_schema
)

model.save('citibike_xgb_model.pkl')